#### Lo primero es importar las librerías que tenemos que usar. En la siguiente celda te explico el por qué de cada una

In [1]:
import requests

## Esta librería no te servirá en este análisis, pero
## es la librería básica para scrapear páginas más sencillas

from bs4 import BeautifulSoup as bs

## Esta sí. Con ella, sacaremos datos del HTML.

import random

## Para usar números aleatorios. Esta no era tan complicada.
## La usaremos para no usar el mismo tiempo siempre al scrapear.

import time

## Con esta metemos el retardo a la máquina. Las máquinas van rápidas. Yo no.

import pandas as pd
import numpy as np

## EL bread and butter del Data Science. Tablas y operaciones aritméticas.

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

## Esta es la manera para usar selenium normal, pero nosotros tenemos que parecer humanos.
## Por eso, usamos la siguiente librería.

import undetected_chromedriver as uc

## La persona que desarrolló esto es un crack. Configuró todos los proxies y
## demás (digo demás porque no se qué brujería ha usado) para ser indetectable.
## Usaremos este browser.

Empecemos iniciando el explorador

In [2]:
browser = uc.Chrome()

TypeError: __init__() got an unexpected keyword argument 'executable_path'

Bien!

Ahora, vayamos a una casa que queramos scrapear. Yo he seleccionado esta, pero pon la que prefieras.

In [ ]:
url = "https://www.idealista.com/inmueble/97786099/"

Vayamos a esa casa

In [ ]:
browser.get(url)

NameError: name 'browser' is not defined

Vaya, hay que pulsar las cookies! No puedes hacer click en nada de la información hasta que no la pulses.

No te preocupes, buscamos el xpath como en el video y hacemos click en él.

(También puedes hacer click manualmente. Nadie se va a enterar)

In [ ]:
browser.find_element("xpath",'//*[@id="didomi-notice-agree-button"]').click()

Ahora, vamos a sacar el html de la página.

In [ ]:
html = browser.page_source
html

Que feo queda no?

Vamos a usar BeautifulSoup para dejarlo mucho mejor. Y apto para sacar info.

In [ ]:
soup = bs(html, 'lxml')
soup

Perfecto, ya podemos empezar a sacarle los datos.

Realizamos lo mismo que aparece en el vídeo.

In [ ]:
titulo = soup.find('span',{'class':'main-info__title-main'}).text
titulo

In [ ]:
localizacion = soup.find('span',{'class':'main-info__title-minor'}).text.split(',')[0]
localizacion

In [ ]:
precio = int(soup.find('span',{'class':'txt-bold'}).text.replace('.',''))
precio

In [ ]:
c1 = soup.find('div',{'class':'details-property'}).find('div',{'class': 'details-property-feature-one'})
c1

In [ ]:
caract_basicas = [caract.text.strip() for caract in c1.find_all('li')]
caract_basicas

In [ ]:
c2 = soup.find('div',{'class':'details-property'}).find('div',{'class': 'details-property-feature-two'})
c2

In [ ]:
caract_extra = [caract.text.strip() for caract in c2.find_all('li')]
caract_extra

In [ ]:
ubicacion = soup.find('div',{'id':'headerMap'})
ubicacion

In [ ]:
ubicacion_desg = [zona.text.strip() for zona in ubicacion.find_all('li')]
ubicacion_desg

Vale, ya hemos sacado todos los datos que queremos!

Lo bueno es, que, si quieres, puedes sacar los datos que tú quieras.

Vamos a automatizar esto para las casas de tu zona.

In [ ]:
url_grande = 'https://www.idealista.com/venta-viviendas/sevilla/los-remedios/'

Bueno sí, sigue siendo mi zona. Pero pon la tuya!

Ahora, queremos sacar el id de cada casa. Vamos a sacar el html de la pagina grande:

In [ ]:
browser.get(url_grande)

In [ ]:
browser.find_element("xpath",'//*[@id="didomi-notice-agree-button"]').click()

In [ ]:
html = browser.page_source

In [ ]:
soup = bs(html, 'lxml')
soup

Ahora localizamos donde está cada casa y las listamos:

In [ ]:
articles = soup.find('main',{'class':'listing-items'}).find_all('article')
articles

Seleccionemos los ids!

In [ ]:
id_muebles = [article.get('data-adid') for article in articles]
id_muebles

Filtramos los nones:

In [ ]:
id_muebles = [muebles for muebles in id_muebles if muebles is not None]
id_muebles

Ahora que tenemos los ids de **una**, toca repetir este proceso para cada página de nuestra zona. Este proceso lo explico muy bien en el video.

In [ ]:
busqueda = 'los remedios'

busqueda = busqueda.replace(' ','-')

x=1
ids = []

while True:
    
    url = f'https://www.idealista.com/venta-viviendas/sevilla/{busqueda}/pagina-{x}.htm'
    
    browser.get(url)
    
    time.sleep(random.randint(10,12))
    
    try:
        
        browser.find_element("xpath",'//*[@id="didomi-notice-agree-button"]').click()
    except:
        pass
    
    html = browser.page_source
    
    soup = bs(html, 'lxml')
    
    pagina_actual = int(soup.find('main',{'class':'listing-items'}).find('div',{'class':'pagination'}).find('li',{'class':'selected'}).text)
    
    if x == pagina_actual:

        articles = soup.find('main',{'class':'listing-items'}).find_all('article')

    else:
        break
    
    x = x+1
    
    for article in articles:
        
        id_muebles = article.get('data-adid')
        
        ids.append(id_muebles)
        
        time.sleep(random.randint(1,3))
        print(id_muebles)
    
    ids = [muebles for muebles in ids if muebles is not None]

In [ ]:
ids_casas = pd.DataFrame(ids)
ids_casas.columns = ['id']
ids_casas

Olé! Vamos a guardar estos ids para no perderlos.

Lo gradaremos en un csv.

Ojo al truqui de index= False!

In [ ]:
ids_casas.to_csv('ids_casas.csv',index = False)

Ahora vamos a usar una técina muy utilizada en programación: crear algo vacío e ir rellenandolo.

En este caso, creamos una serie llamada **casas** a la que vamos a ir añadiendo los datos scrapeados.

Esta serie la pasamos a dataframe y vamos concatenandolas.

In [ ]:
casas = pd.Series()

Aquí tienes la función. He dejado algunos prints comentados, descoméntalos si quieres verlos mientras scrapeamos.

In [ ]:
def parsear_inmueble(id_inmueble):
    
    print( '\n Casa numero: ' + id_inmueble)
    
    url = "https://www.idealista.com/inmueble/" + id_inmueble + "/"
    
    browser.get(url)

    html = browser.page_source
    
    soup = bs(html, 'lxml')

    titulo = soup.find('span',{'class':'main-info__title-main'}).text
    
    print('\n Titulo: ' + titulo)
    
    localizacion = soup.find('span',{'class':'main-info__title-minor'}).text.split(',')[0]

    print('\n Localizacion: ' + localizacion)
    precio = int(soup.find('span',{'class':'txt-bold'}).text.replace('.',''))

    c1 = soup.find('div',{'class':'details-property'}).find('div',{'class': 'details-property-feature-one'})

    caract_basicas = [caract.text.strip() for caract in c1.find_all('li')]
    
    #print('Caracteristicas basicas:' + caract_basicas)

    c2 = soup.find('div',{'class':'details-property'}).find('div',{'class': 'details-property-feature-two'})

    caract_extra = [caract.text.strip() for caract in c2.find_all('li')]
    
    #print('Caracteristicas extras:' + caract_extra)
    
    casas['titulo'] = titulo
    
    casas['localizacion'] = localizacion
    
    casas['precio'] = precio
    
    casas['caracteristicas_basicas'] = caract_basicas
    
    casas['caracteristicas_extras'] = caract_extra
    
    df_casas = pd.DataFrame(casas)
    
    return(df_casas.T)

In [ ]:
df_casas = parsear_inmueble(ids_casas.iloc[0].id)

for i in range(1,len(ids)):
    
    df_casas = pd.concat([df_casas,parsear_inmueble(ids[i])])
    
    time.sleep(random.randint(4,8))

A ver si funciona...

In [ ]:
df_casas

Perfecto! reseteemos el índice y guardemos estos datos en otro csv.

In [ ]:
df_casas.reset_index(drop=True, inplace=True)

In [ ]:
df_casas

In [ ]:
df_casas.to_csv('casas_idealista.csv', index = False, sep = ';', encoding = 'utf-16')

## Hasta aquí el módulo de web scraping.

## Si te ha gustado, comparte el video con amigos y diles cómo conseguir este notebook.

## Si necesitas cualquier cosa, escríbeme al correo: javidatascience@gmail.com. Un saludo y muchas gracias!!